# Car wash queueing model

[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/plugboard-dev/plugboard)

This model uses event-driven components to model cars arriving and queueing at a carwash. We can use the model to understand how many cars are in the queue at a given moment.

Cars arrive at the carwash and enter a queue to go into one of three washing machines. If any one of the machines is empty then the car at the front of the queue moves into it, otherwise they will wait in the queue. The washing process takes a random amount of time (configurable on each machine).

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <p>
        Install <code>pandas</code> and <code>plotly</code> to run this demo.
    </p>
</div>

In [ ]:
from datetime import datetime
import random
import typing as _t

import pandas as pd
from pydantic import BaseModel

from plugboard.connector import AsyncioConnector
from plugboard.component import Component, IOController
from plugboard.connector import AsyncioConnector, ConnectorBuilder
from plugboard.events import Event, EventConnectorBuilder, StopEvent
from plugboard.schemas import ComponentArgsDict, ConnectorSpec
from plugboard.process import LocalProcess
from plugboard.library import FileWriter

Start be defining the events that are required for:
* A car arriving and entering the queue;
* A car moving from the queue into a washing machine;
* A car leaving the car-wash after washing is completed.

In [ ]:
class CarData(BaseModel):
    """Data for a car at the carwash."""

    car_id: int
    machine_id: _t.Optional[int] = None
    arrival_time: _t.Optional[datetime] = None
    leave_time: _t.Optional[datetime] = None


class CarArrived(Event):
    """Event for when a car arrives at the carwash."""

    type: _t.ClassVar[str] = "car_arrived"
    data: CarData


class CarEntersWash(Event):
    """Event for when a car enters the wash."""

    type: _t.ClassVar[str] = "car_enters_wash"
    data: CarData


class CarLeavesWash(Event):
    """Event for when a car leaves the wash."""

    type: _t.ClassVar[str] = "car_leaves_wash"
    data: CarData

The CSV file in`car-arrivals.csv` contains minute-by-minute data on the number of cars that arrive at the carwash. We need a component to read this data and publish an event for each car that arrives.

In [ ]:
class CarArrivals(Component):
    """This component emits an event for each new car that arrives."""

    io = IOController(outputs=["time_stamp"], output_events=[CarArrived])

    def __init__(self, path: str, **kwargs: _t.Unpack[ComponentArgsDict]):
        super().__init__(**kwargs)
        self._path = path
        self._car_id = 0

    async def init(self) -> None:
        df = pd.read_csv(self._path, parse_dates=["time_stamp"])
        self._iterator = df.iterrows()

    async def step(self) -> None:
        try:
            _, row = next(self._iterator)
        except StopIteration:
            self.io.queue_event(StopEvent(source=self.name, data={}))
            return
        self.time_stamp = row["time_stamp"]
        # Emit an event for each car that arrives at this time
        for _ in range(row["cars"]):
            self._car_id += 1
            car = CarData(car_id=self._car_id, arrival_time=self.time_stamp)
            self.io.queue_event(CarArrived(source=self.name, data=car))
            self._logger.info("Car arrived", car=car)

This next component implements a queue. It monitors the status of each washing machine, and publishes events when cars move from the queue to a machine.

In [ ]:
class CarWashQueue(Component):
    """This component manages the queue of cars waiting for the wash."""

    io = IOController(
        outputs=["queue_length"],
        input_events=[CarArrived, CarLeavesWash],
        output_events=[CarEntersWash],
    )

    def __init__(self, n_machines: int = 3, **kwargs: _t.Unpack[ComponentArgsDict]):
        super().__init__(**kwargs)
        self._queue = []
        # Use this dictionary to keep track of which machines are available
        self._machine_status = {idx: False for idx in range(1, n_machines + 1)}

    async def step(self) -> None:
        # Check if there are any cars waiting for the wash
        while self._queue:
            # Check if any of the machines are available
            for idx, machine in self._machine_status.items():
                if not machine:
                    # Machine is not busy - send car to wash
                    car = self._queue.pop(0)
                    car.machine_id = idx
                    self.io.queue_event(CarEntersWash(source=self.name, data=car))
                    self._logger.info("Car enters wash", car=car)
                    # Mark machine as busy
                    self._machine_status[idx] = True
                    break
            else:
                # No machines available
                break
        self.queue_length = len(self._queue)

    @CarArrived.handler
    async def car_arrived(self, event: CarArrived) -> None:
        # Add the car to the queue
        self._queue.append(event.data)

    @CarLeavesWash.handler
    async def car_leaves_wash(self, event: CarLeavesWash) -> None:
        # Mark the machine as available
        self._machine_status[event.data.machine_id] = False

Next model the washing machines. Each one will listen for an event telling it to start washing a car.

In [ ]:
class CarWashMachine(Component):
    """This component simulates the car wash machine."""

    io = IOController(outputs=["busy"], input_events=[CarEntersWash], output_events=[CarLeavesWash])

    def __init__(
        self,
        machine_id: int,
        wash_time_range: tuple[int, int] = (4, 6),
        **kwargs: _t.Unpack[ComponentArgsDict],
    ):
        super().__init__(**kwargs)
        self._machine_id = machine_id
        self._wash_time_range = wash_time_range
        self._time_remaining = 0
        self._car = None
        self.busy = False

    @CarEntersWash.handler
    async def car_enters_wash(self, event: CarEntersWash) -> None:
        # Check if car is going into this machine
        if event.data.machine_id != self._machine_id:
            return
        # Start the wash
        self._time_remaining = random.randint(*self._wash_time_range)
        self.busy = True
        self._car = event.data
        self._logger.info("Car enters wash", car=self._car)

    async def step(self) -> None:
        # Check if the wash is complete
        if self.busy:
            self._time_remaining -= 1
            if self._time_remaining <= 0:
                self.busy = False
                self._car.leave_time = self.time_stamp
                self.io.queue_event(CarLeavesWash(source=self.name, data=self._car))
                self._logger.info("Car leaves wash", car=self._car)

Use this component to capture data as each car leaves the washing machines, so that we can record how long the process took for each vehicle.

In [ ]:
class CaptureData(Component):
    """This component captures the data for each car that leaves the wash and saves to CSV."""

    io = IOController(inputs=["time_stamp"], input_events=[CarLeavesWash])

    def __init__(self, path: str, **kwargs: _t.Unpack[ComponentArgsDict]):
        super().__init__(**kwargs)
        self._path = path
        self._data = []

    async def step(self) -> None:
        pass

    @CarLeavesWash.handler
    async def car_leaves_wash(self, event: CarLeavesWash) -> None:
        car = event.data
        car.leave_time = self.time_stamp
        self._data.append(car.model_dump())
        self._logger.info("Car data captured", car=car)

    async def destroy(self):
        pd.DataFrame(self._data).to_csv(self._path, index=False)
        return await super().destroy()

When building the `Process` to run the model, we need to create connectors both for the normal data inputs/outputs, and also for the events as shown below.

In [ ]:
components = [
    CarArrivals(name="car-arrivals", path="car-arrivals.csv"),
    CarWashQueue(name="car-wash-queue", n_machines=3),
    CarWashMachine(name="car-wash-machine-1", machine_id=1, wash_time_range=(3, 5)),
    CarWashMachine(name="car-wash-machine-2", machine_id=2, wash_time_range=(3, 5)),
    # The third machine takes longer to wash cars
    CarWashMachine(name="car-wash-machine-3", machine_id=3, wash_time_range=(5, 9)),
    CaptureData(name="capture-data", path="car-wash-data.csv"),
    FileWriter(
        name="write-data",
        path="queue-length.csv",
        field_names=[
            "time_stamp",
            "queue_length",
            "machine_1_busy",
            "machine_2_busy",
            "machine_3_busy",
        ],
    ),
]
connect = lambda in_, out_: AsyncioConnector(spec=ConnectorSpec(source=in_, target=out_))
connectors = [
    connect("car-arrivals.time_stamp", "write-data.time_stamp"),
    connect("car-arrivals.time_stamp", "capture-data.time_stamp"),
    connect("car-wash-queue.queue_length", "write-data.queue_length"),
    connect("car-wash-machine-1.busy", "write-data.machine_1_busy"),
    connect("car-wash-machine-2.busy", "write-data.machine_2_busy"),
    connect("car-wash-machine-3.busy", "write-data.machine_3_busy"),
]
connector_builder = ConnectorBuilder(connector_cls=AsyncioConnector)
event_connector_builder = EventConnectorBuilder(connector_builder=connector_builder)
event_connectors = list(event_connector_builder.build(components).values())

In [ ]:
process = LocalProcess(
    components=components,
    connectors=connectors + event_connectors,
)
async with process:
    await process.run()

## Results analysis

Now load the output CSV files and analyse the data. Try going back and adjusting parameters to see their effect.

In [ ]:
df_queue = pd.read_csv("queue-length.csv")
df_data = pd.read_csv("car-wash-data.csv", parse_dates=["arrival_time", "leave_time"])

In [ ]:
# Print the average utilisation of each machine
df_queue[["machine_1_busy", "machine_2_busy", "machine_3_busy"]].mean().to_frame(
    "utilisation_percent"
) * 100

In [ ]:
# Plot the queue length over time
try:
    fig = df_queue.plot(
        backend="plotly",
        x="time_stamp",
        y=["queue_length"],
        title="Queue length at car wash",
        labels={"index": "Time", "value": "Number of cars"},
    )
except (ImportError, ValueError):
    print("Please install plotly to run this cell.")
    fig = None
fig

In [ ]:
# Plot a histogram of the time spent at the car wash for each vehicle
df_data = df_data.assign(
    total_time=(df_data["leave_time"] - df_data["arrival_time"]).dt.total_seconds() / 60
)
try:
    fig = df_data.plot(
        backend="plotly",
        kind="hist",
        x="total_time",
        title="Time spent at car wash",
        labels={"index": "Time", "value": "Number of cars"},
    )
except (ImportError, ValueError):
    print("Please install plotly to run this cell.")
    fig = None
fig